<a href="https://www.kaggle.com/code/aleksandr13254/mysolve?scriptVersionId=109091054" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [149]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm # loops show a smart progress meter
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-cpi-test/CPI_MoM.csv
/kaggle/input/dataset-cpi-test/CPI_YoY.csv
/kaggle/input/dataset-cpi-test/CPI_Russia.csv
/kaggle/input/dataset-cpi-test/Product_Price_Month.csv
/kaggle/input/dataset-cpi-test/Product_Price_Day.csv
/kaggle/input/dataset-cpi-test/Product_CPI.csv
/kaggle/input/dataset/DS_tests/DS_test9(2020-06--2022-09-01).csv
/kaggle/input/dataset/DS_tests/DS_test8(2020-06--2022-08-21).csv
/kaggle/input/dataset/DS_tests/DS_test5(2020-06--2022-07-21).csv
/kaggle/input/dataset/DS_tests/DS_test4(2020-06--2022-07-11).csv
/kaggle/input/dataset/DS_tests/DS_test7(2020-06--2022-08-11).csv
/kaggle/input/dataset/DS_tests/DS_test1(2020-06--2022-06-11).csv
/kaggle/input/dataset/DS_tests/DS_test3(2020-06--2022-07-01).csv
/kaggle/input/dataset/DS_tests/DS_test6(2020-06--2022-08-01).csv
/kaggle/input/dataset/DS_tests/DS_test2(2020-06--2022-06-21).csv
/kaggle/input/dataset/DS_train/DS_train(2020-06--2022-06-01).csv


In [3]:
df = pd.read_csv('../input/dataset/DS_tests/DS_test9(2020-06--2022-09-01).csv', delimiter='\t',parse_dates=['DateObserve'])
# df = pd.read_csv('../input/dataset/DS_train/DS_train(2020-06--2022-06-01).csv', delimiter='\t',parse_dates=['DateObserve'], nrows=107892)
df

,WebPriceId,DateObserve,StockStatus,CurrentPrice
0,1,2020-06-25 19:23:21.010,InStock,49.0
1,1,2020-09-08 07:39:23.593,OutOfStock,NaN
2,1,2020-09-08 15:39:01.663,InStock,49.0
3,1,2020-09-08 23:42:25.007,OutOfStock,NaN
4,1,2020-09-09 07:38:41.163,InStock,49.0
...,...,...,...,...
79401618,10001000,2022-05-31 20:17:28.967,InStock,335.0
79401619,10001000,2022-06-30 19:48:35.817,InStock,214.0
79401620,10001000,2022-07-13 08:10:45.810,InStock,209.0
79401621,10001000,2022-08-16 17:28:18.363,InStock,269.0


In [4]:
df = df[df['StockStatus']=='InStock'][['WebPriceId','DateObserve','CurrentPrice']]
df

,WebPriceId,DateObserve,CurrentPrice
0,1,2020-06-25 19:23:21.010,49.0
2,1,2020-09-08 15:39:01.663,49.0
4,1,2020-09-09 07:38:41.163,49.0
6,1,2020-11-20 16:47:57.243,49.0
8,1,2020-12-20 23:55:45.830,49.0
...,...,...,...
79401618,10001000,2022-05-31 20:17:28.967,335.0
79401619,10001000,2022-06-30 19:48:35.817,214.0
79401620,10001000,2022-07-13 08:10:45.810,209.0
79401621,10001000,2022-08-16 17:28:18.363,269.0


In [5]:
df.dropna()
df

,WebPriceId,DateObserve,CurrentPrice
0,1,2020-06-25 19:23:21.010,49.0
2,1,2020-09-08 15:39:01.663,49.0
4,1,2020-09-09 07:38:41.163,49.0
6,1,2020-11-20 16:47:57.243,49.0
8,1,2020-12-20 23:55:45.830,49.0
...,...,...,...
79401618,10001000,2022-05-31 20:17:28.967,335.0
79401619,10001000,2022-06-30 19:48:35.817,214.0
79401620,10001000,2022-07-13 08:10:45.810,209.0
79401621,10001000,2022-08-16 17:28:18.363,269.0


In [6]:
df['DateObserve'] = pd.to_datetime(df['DateObserve']).apply(lambda x: x.strftime('%Y-%m-%d'))
df

,WebPriceId,DateObserve,CurrentPrice
0,1,2020-06-25,49.0
2,1,2020-09-08,49.0
4,1,2020-09-09,49.0
6,1,2020-11-20,49.0
8,1,2020-12-20,49.0
...,...,...,...
79401618,10001000,2022-05-31,335.0
79401619,10001000,2022-06-30,214.0
79401620,10001000,2022-07-13,209.0
79401621,10001000,2022-08-16,269.0


In [7]:
df = df.groupby(['WebPriceId','DateObserve']).agg({'CurrentPrice': ['mean']}).reset_index()
df.columns = ['Id','Date','Price']
df

,Id,Date,Price
0,1,2020-06-25,49.0
1,1,2020-09-08,49.0
2,1,2020-09-09,49.0
3,1,2020-11-20,49.0
4,1,2020-12-20,49.0
...,...,...,...
40411645,13043141,2022-07-19,2438000.0
40411646,13043142,2022-07-19,2438000.0
40411647,13043143,2022-07-19,2438000.0
40411648,13043144,2022-07-19,2438000.0


In [8]:
df = df[df['Price'] < 100000]
df

,Id,Date,Price
0,1,2020-06-25,49.000000
1,1,2020-09-08,49.000000
2,1,2020-09-09,49.000000
3,1,2020-11-20,49.000000
4,1,2020-12-20,49.000000
...,...,...,...
40411639,13043134,2022-07-21,59.990002
40411640,13043135,2022-07-21,59.990002
40411641,13043136,2022-07-21,99.989998
40411642,13043137,2022-07-21,729.989990


In [9]:
df = df[df['Price'] > 10]
df

,Id,Date,Price
0,1,2020-06-25,49.000000
1,1,2020-09-08,49.000000
2,1,2020-09-09,49.000000
3,1,2020-11-20,49.000000
4,1,2020-12-20,49.000000
...,...,...,...
40411639,13043134,2022-07-21,59.990002
40411640,13043135,2022-07-21,59.990002
40411641,13043136,2022-07-21,99.989998
40411642,13043137,2022-07-21,729.989990


In [10]:
count_filtered = df.groupby(['Id'])['Id'].count()
count_filtered

Id
1            6
2           23
3           22
4           52
5           16
            ..
13043134     1
13043135     1
13043136     1
13043137     1
13043139     1
Name: Id, Length: 11823941, dtype: int64

In [11]:
filtered_ids = pd.DataFrame(count_filtered[count_filtered > 60].index)
filtered_ids

,Id
0,17
1,72
2,73
3,84
4,112
...,...
20289,9682938
20290,9684287
20291,9688017
20292,9709282


In [12]:
df = filtered_ids.merge(df, on='Id',how='inner')
df

,Id,Date,Price
0,17,2020-06-25,354.000000
1,17,2020-07-04,334.000000
2,17,2020-08-07,370.000000
3,17,2020-08-11,347.000000
4,17,2020-08-15,367.000000
...,...,...,...
1622418,10628670,2022-08-17,1113.000000
1622419,10628670,2022-08-20,1126.666667
1622420,10628670,2022-08-24,1004.000000
1622421,10628670,2022-08-29,1101.250000


In [13]:
df.to_csv('Product_Price_Day.csv',sep=';',encoding='utf-8')
# df = pd.read_csv('../input/dataset-cpi-test/Product_Price_Day.csv', delimiter=';', index_col=[0])
df

,Id,Date,Price
0,17,2020-06-25,354.000000
1,17,2020-07-04,334.000000
2,17,2020-08-07,370.000000
3,17,2020-08-11,347.000000
4,17,2020-08-15,367.000000
...,...,...,...
1622418,10628670,2022-08-17,1113.000000
1622419,10628670,2022-08-20,1126.666667
1622420,10628670,2022-08-24,1004.000000
1622421,10628670,2022-08-29,1101.250000


In [14]:
df['Date'] = pd.to_datetime(df['Date']).apply(lambda x: x.strftime('%Y-%m'))
df

,Id,Date,Price
0,17,2020-06,354.000000
1,17,2020-07,334.000000
2,17,2020-08,370.000000
3,17,2020-08,347.000000
4,17,2020-08,367.000000
...,...,...,...
1622418,10628670,2022-08,1113.000000
1622419,10628670,2022-08,1126.666667
1622420,10628670,2022-08,1004.000000
1622421,10628670,2022-08,1101.250000


In [15]:
df = df.groupby(['Id','Date']).agg({'Price': ['mean']}).reset_index()
df.columns = ['Id','Date','Price']
df

,Id,Date,Price
0,17,2020-06,354.000000
1,17,2020-07,334.000000
2,17,2020-08,361.333333
3,17,2020-09,367.500000
4,17,2020-10,390.333333
...,...,...,...
411204,10628670,2022-04,1281.000000
411205,10628670,2022-05,1119.023810
411206,10628670,2022-06,1053.800000
411207,10628670,2022-07,965.785185


In [16]:
df.to_csv('Product_Price_Month.csv',sep=';',encoding='utf-8')
# df = pd.read_csv('../input/dataset-cpi-test/Product_Price_Month.csv', delimiter=';', index_col=[0])
df

,Id,Date,Price
0,17,2020-06,354.000000
1,17,2020-07,334.000000
2,17,2020-08,361.333333
3,17,2020-09,367.500000
4,17,2020-10,390.333333
...,...,...,...
411204,10628670,2022-04,1281.000000
411205,10628670,2022-05,1119.023810
411206,10628670,2022-06,1053.800000
411207,10628670,2022-07,965.785185


In [17]:
date_list_all = df.sort_values(by=['Date']).Date.unique()
date_list_all

array(['2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11',
       '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05',
       '2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11',
       '2021-12', '2022-01', '2022-02', '2022-03', '2022-04', '2022-05',
       '2022-06', '2022-07', '2022-08'], dtype=object)

In [18]:
id_product_list = df['Id'].unique()[:-1]
len(id_product_list)

20293

In [19]:
id_list = list()
date_start_list = list()
date_end_list = list()
cpi_product_list = list()
for id_product in tqdm(id_product_list):
    df_id_product = df.loc[df.Id == id_product]
    date_list = df_id_product.sort_values(by=['Date']).Date.unique()
    for i in range(0, len(date_list)):        
        start_value = df_id_product.loc[df_id_product.Date == date_list[i]].Price.values[0]
        for j in range(i+1, len(date_list)):
            end_value = df_id_product.loc[df_id_product.Date == date_list[j]].Price.values[0]
            cpi_product = end_value/start_value*100
            id_list.append(id_product)
            date_start_list.append(date_list[i])
            date_end_list.append(date_list[j])
            cpi_product_list.append(cpi_product)

100%|██████████| 20293/20293 [35:19<00:00,  9.57it/s] 


In [20]:
cpi_id_product = pd.DataFrame({'Id': id_list, 'Date_start': date_start_list, 'Date_end': date_end_list, 'CPI': cpi_product_list})
cpi_id_product

,Id,Date_start,Date_end,CPI
0,17,2020-06,2020-07,94.350282
1,17,2020-06,2020-08,102.071563
2,17,2020-06,2020-09,103.813559
3,17,2020-06,2020-10,110.263653
4,17,2020-06,2020-11,106.638418
...,...,...,...,...
4116219,9709282,2022-05,2022-07,98.617284
4116220,9709282,2022-05,2022-08,56.022683
4116221,9709282,2022-06,2022-07,85.256983
4116222,9709282,2022-06,2022-08,48.432940


In [21]:
cpi_id_product.to_csv('Product_CPI.csv',sep=';',encoding='utf-8')
# cpi_id_product = pd.read_csv('../input/dataset-cpi-test/Product_CPI.csv', delimiter=';', index_col=[0])
cpi_id_product

,Id,Date_start,Date_end,CPI
0,17,2020-06,2020-07,94.350282
1,17,2020-06,2020-08,102.071563
2,17,2020-06,2020-09,103.813559
3,17,2020-06,2020-10,110.263653
4,17,2020-06,2020-11,106.638418
...,...,...,...,...
4116219,9709282,2022-05,2022-07,98.617284
4116220,9709282,2022-05,2022-08,56.022683
4116221,9709282,2022-06,2022-07,85.256983
4116222,9709282,2022-06,2022-08,48.432940


In [22]:
cpi_id_product = cpi_id_product[cpi_id_product.CPI < 120][cpi_id_product.CPI > 80]
cpi_id_product

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Id,Date_start,Date_end,CPI
0,17,2020-06,2020-07,94.350282
1,17,2020-06,2020-08,102.071563
2,17,2020-06,2020-09,103.813559
3,17,2020-06,2020-10,110.263653
4,17,2020-06,2020-11,106.638418
...,...,...,...,...
4116210,9709282,2022-03,2022-05,105.194379
4116212,9709282,2022-03,2022-07,103.739839
4116218,9709282,2022-05,2022-06,115.670623
4116219,9709282,2022-05,2022-07,98.617284


In [23]:
cpi_count = cpi_id_product[['Id','Date_start','Date_end',]].groupby(['Date_end','Date_start']).count().reset_index()
cpi_count.columns = ['Date_end','Date_start','Count_Id',]
cpi_count

,Date_end,Date_start,Count_Id
0,2020-07,2020-06,6604
1,2020-08,2020-06,9170
2,2020-08,2020-07,8807
3,2020-09,2020-06,8853
4,2020-09,2020-07,8416
...,...,...,...
346,2022-08,2022-03,6203
347,2022-08,2022-04,8371
348,2022-08,2022-05,8277
349,2022-08,2022-06,2692


In [58]:
date_list_year = ['2020', '2021', '2022']
date_list_month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
date_list_year_month = list()
cpi_list = list()
cpi_date_list = list()
for y in date_list_year:
    for m in date_list_month:
        date_list_year_month.append(f'{y}-{m}')
for i in range(1, len(date_list_year_month)):
    item = cpi_count.loc[(cpi_count.Date_start == date_list_year_month[i-1]) & (cpi_count.Date_end == date_list_year_month[i])]
    if item.empty == False:
        cpi_list.append(item.Count_Id.values[0])
        cpi_date_list.append(item.Date_end.values[0])
cpi_id_count = pd.DataFrame({'Date': cpi_date_list, 'Count_Id': cpi_list})
cpi_id_count

,Date,Count_Id
0,2020-07,6604
1,2020-08,8807
2,2020-09,14098
3,2020-10,13261
4,2020-11,12552
5,2020-12,13809
6,2021-01,13611
7,2021-02,15264
8,2021-03,4304
9,2021-04,3962


In [59]:
cpi_df = cpi_id_product.groupby(['Date_start','Date_end']).agg({'CPI': ['mean']}).reset_index()
cpi_df.columns = ['Date_start','Date_end','CPI']
cpi_df

,Date_start,Date_end,CPI
0,2020-06,2020-07,99.851962
1,2020-06,2020-08,99.877480
2,2020-06,2020-09,100.336345
3,2020-06,2020-10,99.870394
4,2020-06,2020-11,99.531039
...,...,...,...
346,2022-05,2022-07,98.602399
347,2022-05,2022-08,98.581517
348,2022-06,2022-07,99.243480
349,2022-06,2022-08,99.277321


In [60]:
date_list_year = ['2020', '2021', '2022']
date_list_month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
date_list_year_month = list()
cpi_list = list()
cpi_date_list = list()
for y in date_list_year:
    for m in date_list_month:
        date_list_year_month.append(f'{y}-{m}')
for i in range(1, len(date_list_year_month)):
    item = cpi_df.loc[(cpi_df.Date_start == date_list_year_month[i-1]) & (cpi_df.Date_end == date_list_year_month[i])]
    if item.empty == False:
        cpi_list.append(item.CPI.values[0])
        cpi_date_list.append(item.Date_end.values[0])
cpi_mom = pd.DataFrame({'Date': cpi_date_list, 'MoM': cpi_list})
cpi_mom

,Date,MoM
0,2020-07,99.851962
1,2020-08,99.337024
2,2020-09,100.570486
3,2020-10,99.910327
4,2020-11,97.395858
5,2020-12,101.445817
6,2021-01,101.144785
7,2021-02,99.449539
8,2021-03,99.582982
9,2021-04,99.711704


In [9]:
# cpi_mom.to_csv('CPI_MoM.csv',sep=';',encoding='utf-8')
cpi_mom = pd.read_csv('../input/dataset-cpi-test/CPI_MoM.csv', delimiter=';', index_col=[0])
cpi_mom

,Date,MoM
0,2020-07,99.851962
1,2020-08,99.337024
2,2020-09,100.570486
3,2020-10,99.910327
4,2020-11,97.395858
5,2020-12,101.445817
6,2021-01,101.144785
7,2021-02,99.449539
8,2021-03,99.582982
9,2021-04,99.711704


In [36]:
cpi_yoy = cpi_mom.copy()
for i in range(12, len(cpi_yoy)):
  temp = 100
  for j in range(11, -1, -1):
    temp *= float(cpi_mom.iloc[i-j, 1])/100
  cpi_yoy.iloc[i, 1] = temp
cpi_yoy = cpi_yoy.iloc[12:, :].reset_index(drop = True)
cpi_yoy.columns = ['Date','YoY']
cpi_yoy

,Date,YoY
0,2021-07,99.079655
1,2021-08,99.651411
2,2021-09,100.461306
3,2021-10,104.488735
4,2021-11,110.284777
5,2021-12,109.804602
6,2022-01,110.881921
7,2022-02,111.630367
8,2022-03,119.660790
9,2022-04,116.932430


In [37]:
# cpi_yoy.to_csv('CPI_YoY.csv',sep=';',encoding='utf-8')
cpi_yoy = pd.read_csv('../input/dataset-cpi-test/CPI_YoY.csv', delimiter=';', index_col=[0])
cpi_yoy = cpi_yoy.iloc[:,1:].reset_index(drop = True)
cpi_yoy

,YoY
0,99.079655
1,99.651411
2,100.461306
3,104.488735
4,110.284777
5,109.804602
6,110.881921
7,111.630367
8,119.660790
9,116.932430


In [128]:
cpi_russia = pd.read_csv('../input/dataset-cpi-test/CPI_Russia.csv', delimiter=';')
cpi_russia = cpi_russia
cpi_russia

,Date,Russia_MoM,Russia_YoY,CFD_MoM,CFD_YoY,Iv_MoM,Iv_YoY
0,2020-07,100.35,103.37,100.36,103.25,100.48,103.92
1,2020-08,99.96,103.58,99.87,103.42,100.02,104.48
2,2020-09,99.93,103.67,99.89,103.48,99.86,104.44
3,2020-10,100.43,103.99,100.38,103.78,100.56,105.06
4,2020-11,100.71,104.42,100.65,104.16,100.73,105.43
5,2020-12,100.83,104.91,100.84,104.70,100.82,105.79
6,2021-01,100.67,105.19,100.79,105.11,101.00,106.34
7,2021-02,100.78,105.67,100.71,105.55,101.03,106.99
8,2021-03,100.66,105.79,100.63,105.71,100.58,106.64
9,2021-04,100.58,105.53,100.54,105.54,100.78,106.53


In [39]:
df_14 = pd.concat([cpi_russia.iloc[12:,1:].reset_index(drop = True), cpi_mom.iloc[12:,1:].reset_index(drop = True), cpi_yoy], axis=1)
df_14

,Russia_MoM,Russia_YoY,CFD_MoM,CFD_YoY,Iv_MoM,Iv_YoY,MoM,YoY
0,100.31,106.46,100.26,106.30,100.31,107.57,100.771324,99.079655
1,100.17,106.68,100.10,106.53,100.29,107.87,99.910265,99.651411
2,100.60,107.40,100.66,107.35,100.53,108.60,101.387850,100.461306
3,101.11,108.13,101.02,108.03,101.51,109.63,103.915668,104.488735
4,100.96,108.40,100.96,108.36,100.86,109.77,102.798455,110.284777
5,100.82,108.39,100.80,108.32,100.97,109.94,101.004126,109.804602
6,100.99,108.73,100.96,108.50,101.22,110.18,102.137140,110.881921
7,101.17,109.15,101.42,109.26,101.13,110.28,100.120816,111.630367
8,107.61,116.69,107.37,116.58,109.02,119.54,106.746746,119.660790
9,101.56,117.83,101.74,117.97,102.40,121.46,97.438199,116.932430


In [41]:
df_13_mom = df_14.iloc[:,[0,-2,-1]].copy()
df_13_mom['Russia_MoM-1'] = df_14['Russia_MoM'].shift(1)
df_13_mom['Russia_YoY-1'] = df_14['Russia_YoY'].shift(1)
df_13_mom = df_13_mom.iloc[1:,:].reset_index(drop = True)
df_13_mom

,Russia_MoM,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,100.17,99.910265,99.651411,100.31,106.46
1,100.60,101.387850,100.461306,100.17,106.68
2,101.11,103.915668,104.488735,100.60,107.40
3,100.96,102.798455,110.284777,101.11,108.13
4,100.82,101.004126,109.804602,100.96,108.40
5,100.99,102.137140,110.881921,100.82,108.39
6,101.17,100.120816,111.630367,100.99,108.73
7,107.61,106.746746,119.660790,101.17,109.15
8,101.56,97.438199,116.932430,107.61,116.69
9,100.12,97.832254,113.383708,101.56,117.83


In [42]:
df_13_yoy = df_14.iloc[:,[1,-2,-1]].copy()
df_13_yoy['Russia_MoM-1'] = df_14['Russia_MoM'].shift(1)
df_13_yoy['Russia_YoY-1'] = df_14['Russia_YoY'].shift(1)
df_13_yoy = df_13_yoy.iloc[1:,:].reset_index(drop = True)
df_13_yoy

,Russia_YoY,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,106.68,99.910265,99.651411,100.31,106.46
1,107.40,101.387850,100.461306,100.17,106.68
2,108.13,103.915668,104.488735,100.60,107.40
3,108.40,102.798455,110.284777,101.11,108.13
4,108.39,101.004126,109.804602,100.96,108.40
5,108.73,102.137140,110.881921,100.82,108.39
6,109.15,100.120816,111.630367,100.99,108.73
7,116.69,106.746746,119.660790,101.17,109.15
8,117.83,97.438199,116.932430,107.61,116.69
9,117.10,97.832254,113.383708,101.56,117.83


In [43]:
df_26 = pd.concat([cpi_russia.iloc[:,1:], cpi_mom.iloc[:,1:]], axis=1)
df_26

,Russia_MoM,Russia_YoY,CFD_MoM,CFD_YoY,Iv_MoM,Iv_YoY,MoM
0,100.35,103.37,100.36,103.25,100.48,103.92,99.851962
1,99.96,103.58,99.87,103.42,100.02,104.48,99.337024
2,99.93,103.67,99.89,103.48,99.86,104.44,100.570486
3,100.43,103.99,100.38,103.78,100.56,105.06,99.910327
4,100.71,104.42,100.65,104.16,100.73,105.43,97.395858
5,100.83,104.91,100.84,104.70,100.82,105.79,101.445817
6,100.67,105.19,100.79,105.11,101.00,106.34,101.144785
7,100.78,105.67,100.71,105.55,101.03,106.99,99.449539
8,100.66,105.79,100.63,105.71,100.58,106.64,99.582982
9,100.58,105.53,100.54,105.54,100.78,106.53,99.711704


In [91]:
df_25 = df_26.iloc[:,[0,-1]].copy()
df_25['Russia_MoM-1'] = df_26['Russia_MoM'].shift(1)
df_25 = df_25.iloc[1:,:].reset_index(drop = True)
df_25

,Russia_MoM,MoM,Russia_MoM-1
0,99.96,99.337024,100.35
1,99.93,100.570486,99.96
2,100.43,99.910327,99.93
3,100.71,97.395858,100.43
4,100.83,101.445817,100.71
5,100.67,101.144785,100.83
6,100.78,99.449539,100.67
7,100.66,99.582982,100.78
8,100.58,99.711704,100.66
9,100.74,100.894241,100.58


In [92]:
scaler = preprocessing.StandardScaler()
names = df_25.columns
df_25_norm = pd.DataFrame(scaler.fit_transform(df_25), columns=names)
df_25_norm

,Russia_MoM,MoM,Russia_MoM-1
0,-0.572304,-0.527321,-0.336815
1,-0.592608,0.087555,-0.604673
2,-0.254207,-0.241532,-0.625278
3,-0.064702,-1.494985,-0.281869
4,0.016514,0.523904,-0.089561
5,-0.091774,0.373840,-0.007143
6,-0.017326,-0.471233,-0.117034
7,-0.098542,-0.404712,-0.041484
8,-0.152687,-0.340545,-0.123902
9,-0.044398,0.248945,-0.178847


In [93]:
df_25_norm_train = df_25_norm.iloc[:-3]
df_25_norm_test = df_25_norm.iloc[-3:]
gb_25 = GradientBoostingRegressor(n_estimators = 200, max_depth = 3)
gb_25.fit(df_25_norm_train.iloc[:, 1:], df_25_norm_train.iloc[:, 0])
df_25_gb_predict = gb_25.predict(df_25_norm.iloc[:, 1:])
df_25_gb_predict

array([-0.57165984, -0.59216484, -0.25425537, -0.06493963,  0.01671039,
       -0.09092903, -0.01830373, -0.0978253 , -0.15202693, -0.04509012,
       -0.07874059, -0.3349132 , -0.43046644, -0.13929548,  0.20611315,
        0.10489401,  0.00947239,  0.12396665,  0.24596516,  4.6048977 ,
        0.51050351, -0.46344519, -0.51758795, -0.41157033, -0.32921356])

In [94]:
gb_25.feature_importances_

array([0.95084568, 0.04915432])

In [95]:
df_25_norm_gb = pd.DataFrame({'Russia_MoM_predict': df_25_gb_predict, 'MoM': df_25_norm.iloc[:, 1], 'Russia_MoM-1': df_25_norm.iloc[:, 2]})
df_25_norm_gb

,Russia_MoM_predict,MoM,Russia_MoM-1
0,-0.571660,-0.527321,-0.336815
1,-0.592165,0.087555,-0.604673
2,-0.254255,-0.241532,-0.625278
3,-0.064940,-1.494985,-0.281869
4,0.016710,0.523904,-0.089561
5,-0.090929,0.373840,-0.007143
6,-0.018304,-0.471233,-0.117034
7,-0.097825,-0.404712,-0.041484
8,-0.152027,-0.340545,-0.123902
9,-0.045090,0.248945,-0.178847


In [96]:
df_25_gb = pd.DataFrame(scaler.inverse_transform(df_25_norm_gb), columns=df_25_norm_gb.columns)
df_25_gb

,Russia_MoM_predict,MoM,Russia_MoM-1
0,99.960952,99.337024,100.35
1,99.930655,100.570486,99.96
2,100.429929,99.910327,99.93
3,100.709649,97.395858,100.43
4,100.830290,101.445817,100.71
5,100.671249,101.144785,100.83
6,100.778556,99.449539,100.67
7,100.661060,99.582982,100.78
8,100.580975,99.711704,100.66
9,100.738978,100.894241,100.58


In [97]:
df_25_norm_train = df_25_norm.iloc[:-3]
df_25_norm_test = df_25_norm.iloc[-3:]
rf_25 = RandomForestRegressor(n_estimators = 100, max_depth = 9)
rf_25.fit(df_25_norm_train.iloc[:, 1:], df_25_norm_train.iloc[:, 0])
df_25_rf_predict = rf_25.predict(df_25_norm.iloc[:, 1:])
df_25_rf_predict

array([-0.43071708, -0.47044539, -0.32398534, -0.14984406, -0.05218147,
       -0.05958118, -0.06613714, -0.09546524, -0.12568222, -0.08304366,
       -0.10172341, -0.23669136, -0.41372934, -0.24452873,  0.10781463,
        0.22117904,  0.0546721 ,  0.07354587,  0.14009811,  3.44918837,
        0.51078282,  0.06957529, -0.42557339, -0.33481418, -0.32066901])

In [98]:
rf_25.feature_importances_

array([0.75460869, 0.24539131])

In [99]:
df_25_norm_rf = pd.DataFrame({'Russia_MoM_predict': df_25_rf_predict, 'MoM': df_25_norm.iloc[:, 1], 'Russia_MoM-1': df_25_norm.iloc[:, 2]})
df_25_norm_rf

,Russia_MoM_predict,MoM,Russia_MoM-1
0,-0.430717,-0.527321,-0.336815
1,-0.470445,0.087555,-0.604673
2,-0.323985,-0.241532,-0.625278
3,-0.149844,-1.494985,-0.281869
4,-0.052181,0.523904,-0.089561
5,-0.059581,0.373840,-0.007143
6,-0.066137,-0.471233,-0.117034
7,-0.095465,-0.404712,-0.041484
8,-0.125682,-0.340545,-0.123902
9,-0.083044,0.248945,-0.178847


In [100]:
df_25_rf = pd.DataFrame(scaler.inverse_transform(df_25_norm_rf), columns=df_25_norm_rf.columns)
df_25_rf

,Russia_MoM_predict,MoM,Russia_MoM-1
0,100.169200,99.337024,100.35
1,100.110500,100.570486,99.96
2,100.326900,99.910327,99.93
3,100.584200,97.395858,100.43
4,100.728500,101.445817,100.71
5,100.717567,101.144785,100.83
6,100.707880,99.449539,100.67
7,100.664547,99.582982,100.78
8,100.619900,99.711704,100.66
9,100.682900,100.894241,100.58


In [101]:
scaler = preprocessing.StandardScaler()
names = df_13_mom.columns
df_13_norm = pd.DataFrame(scaler.fit_transform(df_13_mom), columns=names)
df_13_norm

,Russia_MoM,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,-0.448931,-0.365105,-1.907280,-0.418912,-1.111940
1,-0.233336,0.224650,-1.760327,-0.490500,-1.060631
2,0.022369,1.233588,-1.029559,-0.270621,-0.892710
3,-0.052838,0.787670,0.022118,-0.009834,-0.722456
4,-0.123032,0.071492,-0.065008,-0.086536,-0.659486
5,-0.037797,0.523717,0.130469,-0.158124,-0.661818
6,0.052452,-0.281067,0.266272,-0.071195,-0.582522
7,3.281359,2.363568,1.723373,0.020847,-0.484568
8,0.247992,-1.351791,1.228319,3.313926,1.273941
9,-0.474000,-1.194510,0.584412,0.220273,1.539817


In [103]:
df_13_norm_train = df_13_norm.iloc[:-3]
df_13_norm_test = df_13_norm.iloc[-3:]
gb_13 = GradientBoostingRegressor(n_estimators = 200, max_depth = 3)
gb_13.fit(df_13_norm_train.iloc[:, 1:], df_13_norm_train.iloc[:, 0])
df_13_gb_predict = gb_13.predict(df_13_norm.iloc[:, 1:])
df_13_gb_predict

array([-0.44893065, -0.2333359 ,  0.02236934, -0.05283807, -0.1230316 ,
       -0.03779657,  0.05245231,  3.28135894,  0.24799176, -0.47399973,
       -0.19123672, -0.23413752, -0.23413752])

In [104]:
gb_13.feature_importances_

array([0.51829099, 0.46960066, 0.00353545, 0.00857289])

In [105]:
df_13_norm_gb = pd.DataFrame({'Russia_MoM_predict': df_13_gb_predict, 'MoM': df_13_norm.iloc[:, 1], 'YoY': df_13_norm.iloc[:, 2], 'Russia_MoM-1': df_13_norm.iloc[:, 3], 'Russia_YoY-1': df_13_norm.iloc[:, 4]})
df_13_norm_gb

,Russia_MoM_predict,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,-0.448931,-0.365105,-1.907280,-0.418912,-1.111940
1,-0.233336,0.224650,-1.760327,-0.490500,-1.060631
2,0.022369,1.233588,-1.029559,-0.270621,-0.892710
3,-0.052838,0.787670,0.022118,-0.009834,-0.722456
4,-0.123032,0.071492,-0.065008,-0.086536,-0.659486
5,-0.037797,0.523717,0.130469,-0.158124,-0.661818
6,0.052452,-0.281067,0.266272,-0.071195,-0.582522
7,3.281359,2.363568,1.723373,0.020847,-0.484568
8,0.247992,-1.351791,1.228319,3.313926,1.273941
9,-0.474000,-1.194510,0.584412,0.220273,1.539817


In [106]:
df_13_gb = pd.DataFrame(scaler.inverse_transform(df_13_norm_gb), columns=df_13_norm_gb.columns)
df_13_gb

,Russia_MoM_predict,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,100.170000,99.910265,99.651411,100.31,106.46
1,100.600000,101.387850,100.461306,100.17,106.68
2,101.110000,103.915668,104.488735,100.60,107.40
3,100.960000,102.798455,110.284777,101.11,108.13
4,100.820000,101.004126,109.804602,100.96,108.40
5,100.990000,102.137140,110.881921,100.82,108.39
6,101.170000,100.120816,111.630367,100.99,108.73
7,107.610000,106.746746,119.660790,101.17,109.15
8,101.560000,97.438199,116.932430,107.61,116.69
9,100.120000,97.832254,113.383708,101.56,117.83


In [107]:
df_13_norm_train = df_13_norm.iloc[:-3]
df_13_norm_test = df_13_norm.iloc[-3:]
rf_13 = RandomForestRegressor(n_estimators = 100, max_depth = 9)
rf_13.fit(df_13_norm_train.iloc[:, 1:], df_13_norm_train.iloc[:, 0])
df_13_rf_predict = rf_13.predict(df_13_norm.iloc[:, 1:])
df_13_rf_predict

array([-0.35376814, -0.23484009, -0.05238681, -0.05273778, -0.07429725,
       -0.04872671, -0.03428688,  2.04855842,  0.60141663, -0.18044003,
       -0.10563369, -0.10182318, -0.09866446])

In [108]:
rf_13.feature_importances_

array([0.37179591, 0.36610986, 0.08487475, 0.17721947])

In [109]:
df_13_norm_rf = pd.DataFrame({'Russia_MoM_predict': df_13_rf_predict, 'MoM': df_13_norm.iloc[:, 1], 'YoY': df_13_norm.iloc[:, 2], 'Russia_MoM-1': df_13_norm.iloc[:, 3], 'Russia_YoY-1': df_13_norm.iloc[:, 4]})
df_13_norm_rf

,Russia_MoM_predict,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,-0.353768,-0.365105,-1.907280,-0.418912,-1.111940
1,-0.234840,0.224650,-1.760327,-0.490500,-1.060631
2,-0.052387,1.233588,-1.029559,-0.270621,-0.892710
3,-0.052738,0.787670,0.022118,-0.009834,-0.722456
4,-0.074297,0.071492,-0.065008,-0.086536,-0.659486
5,-0.048727,0.523717,0.130469,-0.158124,-0.661818
6,-0.034287,-0.281067,0.266272,-0.071195,-0.582522
7,2.048558,2.363568,1.723373,0.020847,-0.484568
8,0.601417,-1.351791,1.228319,3.313926,1.273941
9,-0.180440,-1.194510,0.584412,0.220273,1.539817


In [110]:
df_13_rf = pd.DataFrame(scaler.inverse_transform(df_13_norm_rf), columns=df_13_norm_rf.columns)
df_13_rf

,Russia_MoM_predict,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,100.3598,99.910265,99.651411,100.31,106.46
1,100.5970,101.387850,100.461306,100.17,106.68
2,100.9609,103.915668,104.488735,100.60,107.40
3,100.9602,102.798455,110.284777,101.11,108.13
4,100.9172,101.004126,109.804602,100.96,108.40
5,100.9682,102.137140,110.881921,100.82,108.39
6,100.9970,100.120816,111.630367,100.99,108.73
7,105.1512,106.746746,119.660790,101.17,109.15
8,102.2649,97.438199,116.932430,107.61,116.69
9,100.7055,97.832254,113.383708,101.56,117.83


In [184]:
fig1 = go.Figure()
fig1.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[:, 0], y=df_25_gb.iloc[:, 0], line_color="blue", name='Градиентный бустинг',)
)
fig1.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[:, 0], y=df_25_rf.iloc[:, 0], line_color="green", name='Случайный лес')
)
fig1.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[:, 0], y=df_25.iloc[:, 0], line_color="red", name='Фактические данные')
)
fig1.update_layout(title_text='<b>Прогнозы инфляция, месяц к месяцу в %<b>')

In [185]:
fig2 = go.Figure()
fig2.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[-len(df_13_gb):, 0], y=df_13_gb.iloc[:, 0], line_color="blue", name='Градиентный бустинг')
)
fig2.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[-len(df_13_gb):, 0], y=df_13_rf.iloc[:, 0], line_color="green", name='Случайный лес')
)
fig2.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[-len(df_13_gb):, 0], y=df_13_mom.iloc[:, 0], line_color="red", name='Фактические данные')
)
fig2.update_layout(title_text='<b>Прогнозы инфляция, месяц к месяцу в %<b>')

In [123]:
mean_absolute_error(df_25.iloc[-3:, 0], df_25_gb.iloc[-3:, 0])

0.6058367655809889

In [124]:
mean_absolute_error(df_25.iloc[-3:, 0], df_25_rf.iloc[-3:, 0])

0.6931666666666653

In [126]:
mean_absolute_error(df_13_mom.iloc[-3:, 0], df_13_gb.iloc[-3:, 0])

1.0469229098899386

In [127]:
mean_absolute_error(df_13_mom.iloc[-3:, 0], df_13_rf.iloc[-3:, 0])

1.2818666666666634

In [186]:
fig3 = px.scatter(x=df_25.iloc[:, 0], y=df_25.iloc[:, 1])
fig3.update_layout(title_text='<b>Прогнозы инфляция, месяц к месяцу в %<b>', 
                   xaxis_title="Инфляция товаров из онлайн магазинов", 
                   yaxis_title="Инфляция на товары и услуги в России")
fig3.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

In [187]:
fig3 = px.scatter(x=df_25.iloc[:, 0], y=df_25.iloc[:, 2])
fig3.update_layout(title_text='<b>Прогнозы инфляция, месяц к месяцу в %<b>', 
                   xaxis_title="Инфляция на товары и услуги в России за прошлый месяц", 
                   yaxis_title="Инфляция на товары и услуги в России")
fig3.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))